# GTEP Model Demo

This notebook introduces the Generation and Transmission Expansion Planning (GTEP) model.  
It explains the required input format, presents the mathematical formulation, and demonstrates how 
to run the optimization. It also covers spatial and temporal deaggregation of solutions.  

**Purpose**: Show how to set up and run the GTEP model on aggregated input data, and how to recover 
fine-scale outputs.  
**Inputs**: CSV input datasets (not necessarily from the aggregation pipeline)  
**Outputs**: Optimization results and deaggregated results under `<Your_Specified_Path>/results/`  

In [1]:
import sys
import pathlib

PROJECT_ROOT = pathlib.Path().cwd().parent
sys.path.append(str(PROJECT_ROOT))

## 1) What the aggregated GTEP expects as input

The **GTEP module can be used independently**. It expects data in the following shapes (pandas DataFrames):

- **Buses**: columns `['bus_id', 'Lat', 'Lon']`, one row per bus.
- **Branches**: columns `['branch_id','from_bus_id','to_bus_id']`, one row per line.
- **RepPeriod** (time series):
    - `Load`: shape **T × B** (hours × buses), numeric
    - `PV`: shape **T × B**, numeric (capacity factors or per-bus availability factors)
    - `Wind`: shape **T × B**, numeric

In [3]:
from src.gtep.types import RepPeriod, InputData, Params
import pandas as pd
import numpy as np

# build your own (toy) dataframes:
B, T = 5, 24*10
buses = pd.DataFrame({"bus_id": range(B), "Lat": np.linspace(40,45,B), "Lon": np.linspace(-73,-70,B)})
branches = pd.DataFrame(
    [{"branch_id": i*B+j, "from_bus_id": i, "to_bus_id": j}
     for i in range(B) for j in range(i+1, B)]
)
Load = pd.DataFrame(100 + 20*np.random.rand(T, B), columns=buses.bus_id)
PV   = pd.DataFrame(np.clip(np.random.rand(T, B), 0, 1), columns=buses.bus_id)
Wind = pd.DataFrame(np.clip(np.random.rand(T, B), 0, 1), columns=buses.bus_id)

# Use the utils to scale and package
params = Params(
    Initial_SOC=0.5, eta_discharge=0.92, eta_charge=0.92,
    PV_resource=1e5, Wind_resource=5e4,
    CCGT_ramp=0.05, CCGT_max_cap=640.0,
    c_ccgt=1.0, c_pv=1.0, c_wind=1.0, c_storage=1.0, c_tran=1.0,
    d_ccgt=1.0, d_shed=100.0,
    rate_duration=4, MW_per_mile=30.0,
    TranMax={i:1000.0 for i in branches.index}, Length={i:10.0 for i in branches.index}
)
data = InputData(buses=buses, branches=branches, rep_period=RepPeriod(Load, PV, Wind), params=params)
print("Buses:", data.buses.shape, "Branches:", data.branches.shape, "T:", data.rep_period.T)

Buses: (5, 3) Branches: (10, 3) T: 240


Alternatively, you can pass an **aggregation block** to `build_input_data(...)`, which converts it into the exact `InputData` structure.

In [7]:
from src.aggregation.pipeline import StaticPreprocessor, DynamicProcessor
from src.gtep.utils import build_input_data

# We get "agg_spatiotemporal" from the aggregation pipeline:
preproc = StaticPreprocessor(granularity="coarse", year=2013).preprocess()
dyn = DynamicProcessor(preprocessor=preproc)

weights={
        'position': 1.0,
        'time_series': 0.8,
        'duration_curves': 1.2,
        'ramp_duration_curves': 1.0,
        'intra_correlation': 1.0,
        'inter_correlation': 1.0
    }

results, version_hash, day_weights, metadata = dyn.run_with_hyperparameters(
    weights=weights,
    n_representative_nodes=10,
    k_representative_days=12
)

agg_spatiotemporal = results["spatiotemporal"]

# We can now build the input data for the GTEP model
data = build_input_data(agg_spatiotemporal, power_scale="GW", cost_scale="M$", sig_round=3, verbose=True)
print("Buses:", data.buses.shape, "Branches:", data.branches.shape, "T:", data.rep_period.T)

c:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\src\aggregation\utils.py:140: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties.geometry.centroid.y,
c:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\src\aggregation\utils.py:141: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties.geometry.centroid.x


Cached metrics loaded from C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\distance_metrics\v202586c2.
Results saved to C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\joint_aggregation_results\v4b02ae39
=== DATA LOADING ===
PV shape  (288, 10)
Wind shape  (288, 10)
Load shape  (288, 10)


=== SCALING CHECK ===
Power: MW → GW (×0.001), Cost: $ → M$ (×1e-06)
    PV CF range: [0, 0.866]
    Wind CF range: [0, 1]
    Load range MW → GW:  [605, 1.04e+04] → [0.605, 10.4]
    Param range: [30.6, 2.36e+06] → [0.0306, 2.36e+03]

Buses: (10, 3) Branches: (45, 3) T: 288


## 2) Model description

The **aggregated Generation and Transmission Expansion Planning (GTEP) model** is formulated as a transport-based MILP. The model co-optimizes investment and operation decisions across representative hours and aggregated zones. It captures demand, renewable profiles, storage dynamics, and transmission flows, while enforcing simplified operational limits.

### Sets and parameters

| **Symbol** | **Meaning** |
|:--:|:--:|
| $\mathcal{C}$ | Set of zones (clusters) |
| $\mathcal{T}_{\text{agg}}$ | Set of representative time steps $t$ with weight $\omega_t$ |
| $\mathcal{G}_{\text{disp}}$ | Dispatchable technologies (e.g. gas) |
| $\mathcal{G}_{\text{var}}$ | Variable renewables (e.g. wind, solar) |
| $\mathcal{E}$ | Transmission links $(c,c')$ |
| $D^d_c(t)$ | Demand in zone $c$ at time $t$ |
| $a_c^g(t)$ | Availability factor of renewable $g$ in $c$ at $t$ |
| $\bar{P}_g$ | Per-unit capacity of technology $g$ |
| $r_g$ | Ramp rate of dispatchable $g$ |
| $\Gamma_g^{\max}$ | Max installable potential of renewable $g$ |
| $\eta^{ch}, \eta^{dis}$ | Storage charge/discharge efficiency |
| $\tau$ | Max storage duration (energy-to-power ratio) |


### Decision variables

**Investments (per zone $c$):**
- $G_c^g$: capacity of technology $g$ (integer if dispatchable, real if renewable)  
- $S_c$: storage power capacity  
- $T_{cc'}$: transmission capacity on link $(c,c')$

**Operations (per $t$):**
- $P_c^g(t)$: generation dispatch  
- $q_c^g(t)$: curtailment of renewable $g$  
- $\ell_c(t)$: load shedding  
- $f_{cc'}(t)$: flow on link $(c,c’)$  
- $C_c(t), D_c(t)$: storage charge and discharge  
- $e_c(t)$: storage energy level  

### Constraints

**Power balance (per zone, per $t$):**
$$
\sum_g P_c^g(t) + D_c(t) - C_c(t) 
+ \sum_{c'} f_{c'c}(t) - \sum_{c'} f_{cc'}(t) 
+ \ell_c(t) = D^d_c(t)
$$

**Generation capacity and renewables:**
$$
P_c^g(t) \le G_c^g \cdot \bar{P}_g \quad (g \in \mathcal{G}_{\text{disp}})
$$
$$
P_c^g(t) + q_c^g(t) = G_c^g \cdot a_c^g(t) \quad (g \in \mathcal{G}_{\text{var}})
$$

**Ramping (dispatchables):**
$$
|P_c^g(t) - P_c^g(t-1)| \le G_c^g \cdot \bar{P}_g \cdot r_g
$$

**Storage dynamics:**
$$
e_c(t) = e_c(t-1) + \eta^{ch} C_c(t) - \tfrac{1}{\eta^{dis}} D_c(t)
$$
$$
0 \le C_c(t), D_c(t) \le S_c, \quad 0 \le e_c(t) \le \tau S_c
$$

**Transmission limits:**
$$
- T_{cc'} \le f_{cc'}(t) \le T_{cc'} \quad \forall (c,c')
$$

**Resource potentials:**
$$
\sum_c G_c^g \le \Gamma_g^{\max} \quad (g \in \mathcal{G}_{\text{var}})
$$

### Objective

The objective minimizes investment and weighted operating costs:
$$
\min \ \sum_{c,g} C_g^{inv} G_c^g
+ \sum_c C_S^{inv} S_c
+ \sum_{(c,c')} C_T^{inv} T_{cc'}
+ \sum_{t \in \mathcal{T}_{\text{agg}}} \omega_t \sum_c 
\Big( \sum_{g \in \mathcal{G}_{\text{disp}}} \varphi_g P_c^g(t) + \psi \ell_c(t) \Big)
$$

This formulation treats dispatchable investments as integer (discrete units) and all other capacities as continuous, ensuring tractability while capturing essential planning trade-offs.

Results are stored under the folders: `<Your_Specified_Path>/results/gtep_output/aggregated/` in pickle format `aggregated_<aggregation_hash>_<gtep_hash>.pkl`, as a RunArtifacts dataclass, defined in `src/gtep/types.py`.

---
You can either use the provided `run_aggregated` helper function that builds `InputData` for the GTEP based on the aggregation pipeline results and runs the aggregated GTEP:

In [8]:
from pathlib import Path
from src.gtep.pipeline import run_aggregated

day_weights_list = list(day_weights.values())
# You can also use a dummy uniform day-weights if you built your own data:
# from gtep.utils import uniform_day_weights
# day_weights = uniform_day_weights(data.rep_period.T)

# Running Aggregated GTEP:
exp_root = Path(".")
art_agg = run_aggregated(results, agg_hash=version_hash, exp_root=exp_root, day_weights=day_weights_list, threads=4)
print("Saved:", art_agg.save_path, "Log:", art_agg.log_path)


=== DATA LOADING ===
PV shape  (288, 10)
Wind shape  (288, 10)
Load shape  (288, 10)


=== SCALING CHECK ===
Power: MW → GW (×0.001), Cost: $ → M$ (×1e-06)
    PV CF range: [0, 0.866]
    Wind CF range: [0, 1]
    Load range MW → GW:  [605, 1.04e+04] → [0.605, 10.4]
    Param range: [30.6, 2.36e+06] → [0.0306, 2.36e+03]

Set parameter Username
Set parameter TimeLimit to value 7200
Set parameter Threads to value 4
Academic license - for non-commercial use only - expires 2026-08-14
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  7200
Threads  4

Optimize a model with 49007 rows, 30325 columns and 147928 nonzeros
Model fingerprint: 0x5b6e1e10
Variable types: 30315 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+00]
  Obj

Or you can use your own data, transform it yourself in `InputData` as seen in part 1), and manually run the `AggregatedGTEP` (NB, you will still need to have the cluster medoids and members):

In [10]:
import gurobipy as gp
import hashlib
import json
from dataclasses import asdict
from src.gtep.models import AggregatedGTEP
from src.gtep.types import RunArtifacts
from src.gtep.utils import SolveLogger
from src.gtep.pipeline import save_artifacts

# See part 1) of this notebook to build the data
agg_hash = version_hash # from the aggregation pipeline, or a Dummy Hash
agg_results = results
day_weights_list = list(day_weights.values())

# Manually set parameters and solve the aggregated GTEP
threads = 4
gtep_hash = hashlib.md5(json.dumps(asdict(data.params), sort_keys=True).encode()).hexdigest()[:8]
log_path = exp_root / "results" / "gtep_logs" / "aggregated" / f"aggregated_{agg_hash}_{gtep_hash}.csv"
clusters = agg_results["clusters"]
with gp.Env(empty=True) as env:
    env.setParam('OutputFlag', 1)
    env.setParam("Threads",   threads)
    env.setParam("TimeLimit", 2 * 60 * 60) # in seconds
    env.start()
    logger = SolveLogger(log_path)
    model = AggregatedGTEP(env, data.buses, data.branches, data.params)
    try:
        res = model.optimize(data.rep_period, day_weights_list, logger)
    except RuntimeError as e:
        if "No feasible solution" in str(e):
            print(f"[{agg_hash}] Infeasible aggregated GTEP → skipping")
            res = None
        else:
            raise
art_agg = RunArtifacts(
    results=res,
    data=data,
    agg_hash=agg_hash,
    gtep_hash=gtep_hash,
    save_path=None,
    log_path=log_path,
    clusters=clusters
)
if res is not None:
    art_agg.save_path = save_artifacts(art_agg, "aggregated", exp_root)

Set parameter Username
Set parameter TimeLimit to value 7200
Set parameter Threads to value 4
Academic license - for non-commercial use only - expires 2026-08-14
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  7200
Threads  4

Optimize a model with 49007 rows, 30325 columns and 147928 nonzeros
Model fingerprint: 0x5b6e1e10
Variable types: 30315 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+00]
  Objective range  [3e-01, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 2e+03]
Presolve removed 85 rows and 2910 columns
Presolve time: 0.15s
Presolved: 48922 rows, 27415 columns, 144913 nonzeros
Variable types: 27405 continuous, 10 integer (0 binary)

Deterministic concurrent LP optimizer: primal and du

## 4) Spatial & temporal de-aggregation

We evaluate aggregated solutions using a **two-stage de-aggregation procedure**. This ensures feasibility under the same physical and operational rules as the full GTEP model.

### 1. Spatial de-aggregation

For each cluster $c$ and generation technology $g$, node-level investments must reproduce the aggregated totals:

$$
\sum_{i \in c} G_i^g = G_c^g, 
\quad \sum_{i \in c} S_i = S_c
$$

Transmission is treated similarly. For each aggregated link $(c,c')$, with set of physical lines  
$\mathcal{E}_{cc'} = \{(i,j): i \in c,\, j \in c'\}$, we impose:

$$
\sum_{(i,j)\in \mathcal{E}_{cc'}} T_{ij} = T_{cc'}
$$

These equalities preserve the aggregated capacities while letting the optimizer decide their distribution across nodes and lines. All other operational constraints remain the same as in the aggregated model.

### 2. Temporal de-aggregation

After spatial allocation, investments are **fixed**, and the operational MILP is re-solved over the full chronological horizon $\mathcal{T} = \{1, \dots, 8760\}$ with **uniform day weights**.  

The objective minimizes real operating costs and penalties for load shedding, subject to dispatch, storage, and flow constraints. This step tests whether the aggregated plan remains feasible under chronological flexibility requirements.

### Outputs

Results are stored in the same format as the aggregated runs, but under the folders:

- `<Your_Specified_Path>/results/gtep_output/spatial/` for Stage 1 (spatial allocation)  
- `<Your_Specified_Path>/results/gtep_output/temporal/` for Stage 2 (full chronological operation)

In [14]:
from src.gtep.pipeline import run_spatial, run_temporal

# Given aggregated results/artifacts:
if art_agg.results:
    art_sp = run_spatial(results, agg_hash=version_hash, exp_root=exp_root, day_weights=day_weights_list, agg_inv=art_agg.results.inv, threads=4)
    if art_sp.results:
        art_tm = run_temporal(results, agg_hash=version_hash, exp_root=exp_root, fixed_inv=art_sp.results.inv, threads=4)
print("Artifacts saved under <Your_Specified_Path>/results/gtep_output/{spatial|temporal}/")


=== DATA LOADING ===
PV shape  (288, 17)
Wind shape  (288, 17)
Load shape  (288, 17)


=== SCALING CHECK ===
Power: MW → GW (×0.001), Cost: $ → M$ (×1e-06)
    PV CF range: [0, 0.866]
    Wind CF range: [0, 1]
    Load range MW → GW:  [449, 5.83e+03] → [0.449, 5.83]
    Param range: [30.6, 2.36e+06] → [0.0306, 2.36e+03]

Set parameter Username
Set parameter TimeLimit to value 14400
Set parameter Threads to value 4
Academic license - for non-commercial use only - expires 2026-08-14
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  14400
Threads  4

Optimize a model with 117727 rows, 68748 columns and 354540 nonzeros
Model fingerprint: 0x2f3f3391
Variable types: 68731 continuous, 17 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+00]
  

## 5) Running the pipeline end-to-end (parallel vs simple)

Use `run_experiment` to iterate over multiple (N, K, weights) cases.

- Set `threads` to control Gurobi’s parallelism per run.
- Use `consume(...)` to build a manifest of results (hashes, paths, objectives).

In [16]:
from src.gtep.pipeline import run_experiment, consume

# 1) Build aggregated data (choose "coarse" 17 zones OR "fine" 385 nodes) from part 1) of this notebook
# pre = StaticPreprocessor(granularity="coarse", year=2013).preprocess()
# dyn = DynamicProcessor(preprocessor=pre)

# 2) Define a tiny grid of hyperparameters
grid = [
    {"id":"case_01","desc":"C10-D12","n_representative_nodes":10,"k_representative_days":12,
     "weights":{"position":0.4,"time_series":0.4,"duration_curves":0.2,"ramp_duration_curves":0.2,"intra_correlation":0.1,"inter_correlation":0.4}},
    {"id":"case_02","desc":"C12-D16","n_representative_nodes":12,"k_representative_days":16,
     "weights":{"position":0.4,"time_series":0.4,"duration_curves":0.2,"ramp_duration_curves":0.2,"intra_correlation":0.1,"inter_correlation":0.4}},
]

# 3) Run GTEP pipeline
exp_root = Path(".")
gen = run_experiment(grid, dyn, exp_root=exp_root, threads=8)

# 4) Collect results into a manifest
manifest = consume(gen, root=exp_root)
print(manifest)


=== Starting case case_01 (N=10, K=12) ===
[case_01] Running aggregation...
Cached metrics loaded from C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\distance_metrics\v202586c2.
Results saved to C:\Users\g630d\Documents\00_Academic\2024-2025_MIT\Research\2024 09 Thesis\Code\results\joint_aggregation_results\v8a7dce83
[case_01] Aggregation done. agg_hash=8a7dce83
[case_01] Solving aggregated GTEP...
=== DATA LOADING ===
PV shape  (288, 10)
Wind shape  (288, 10)
Load shape  (288, 10)


=== SCALING CHECK ===
Power: MW → GW (×0.001), Cost: $ → M$ (×1e-06)
    PV CF range: [0, 0.866]
    Wind CF range: [0, 1]
    Load range MW → GW:  [567, 1.04e+04] → [0.567, 10.4]
    Param range: [30.6, 2.36e+06] → [0.0306, 2.36e+03]

Set parameter Username
Set parameter TimeLimit to value 7200
Set parameter Threads to value 8
Academic license - for non-commercial use only - expires 2026-08-14
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 